<a href="https://colab.research.google.com/github/Arch0290/BeZenAssignment/blob/main/BeZen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Be Zen Assignment

This assignment attempts to solve the queries given in the assignment.

The first section deals with exploring the data

In [27]:
#imprting the libraries

import pandas as pd
import numpy as np

In [28]:
#Reading the csv file

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/2022_02_08-02_30_31_AM.csv")
df.head()

,uuid,price_string,price_string_unf,product_type,level_1,category
0,638744a4-b0ae-4166-8cb6-5c063c862036,NaN,NaN,TGFwdG9wIENvdmVycy9CYWdz,TGFwdG9wIENhc2U,RWxlY3Ryb25pY3M
1,ab313969-02cc-48b2-9daf-0054efb70b92,NaN,NaN,QmFraW5nIEN1cHMgYW5kIE1hdHM,QmFraW5nIE1hdHMgLyBCYWtpbmcgZGlzaA,a2l0Y2hpbmcgYW5kIGRpbmluZw
2,acbd66ff-79f8-467a-91f9-108a45af5626,$19.95,NaN,R3VtbWllcyB2aXRhbWlucyBhbmQgbWluZXJhbHMgZm9yIG...,SW1tdW5pdHkgZ3VtbWllcw,SGVhbHRo
3,963915d6-b2e3-4098-b242-9410593cf205,$92.00,NaN,U2VydW1z,RmFjZSBTZXJ1bQ,YmVhdXR5IGFuZCBwZXJzb25hbCBjYXJl
4,b5b68f3c-b1e0-40e5-8ee5-e2f7236c1daf,11.50,NaN,RWF0aW5nIFV0ZW5zaWxzL0N1dGxlcnk,Q2hvcHN0aWNrcw,a2l0Y2hpbmcgYW5kIGRpbmluZw


In [29]:
df.info()
print("\n Exploring number of null elements in the dataset \n", df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474697 entries, 0 to 474696
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   uuid              474697 non-null  object
 1   price_string      190611 non-null  object
 2   price_string_unf  10747 non-null   object
 3   product_type      474345 non-null  object
 4   level_1           472988 non-null  object
 5   category          474345 non-null  object
dtypes: object(6)
memory usage: 21.7+ MB

 Exploring number of null elements in the dataset 
 uuid                     0
price_string        284086
price_string_unf    463950
product_type           352
level_1               1709
category               352
dtype: int64


#Observation

- The dataset contains 6 columns describing a product
- It contains null values that needs to be explored

Q1. To find the Products without prices

- The price_string column has 284086 null elements.

In [31]:
print("Total products without prices = ", df['price_string'].isnull().sum())

Total products without prices =  284086


In [30]:
df1 = df.copy(deep=True)
NaN_prices_product = df1[df1['price_string'].isnull()]
df2 = NaN_prices_product.iloc[:, 0:2]

In [31]:
print(df2.info())

df2.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284086 entries, 0 to 474696
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   uuid          284086 non-null  object
 1   price_string  0 non-null       object
dtypes: object(2)
memory usage: 6.5+ MB
None


,uuid,price_string
0,638744a4-b0ae-4166-8cb6-5c063c862036,NaN
1,ab313969-02cc-48b2-9daf-0054efb70b92,NaN
5,389d9f75-cc3f-4bd2-94f7-93e381a3bed5,NaN
13,1f2766ec-0a27-43f8-b28b-77a44492ead1,NaN
16,7bad39f5-74b9-4611-afb9-46fb135190cd,NaN


In [39]:
len(df2['uuid'].unique())

822

Q2. Count of products without prices and with prices in each Product Type, Category, Level 1

In [36]:
df.groupby('category')['price_string'].value_counts()

category                          price_string
Q2xvdGhpbmcgYW5kIEFjY2Vzc29yaWVz  $89             3893
                                  $160            2476
                                  $199            2086
                                  $79             2080
                                  $75.00          1959
                                                  ... 
cGV0IHN1cHBsaWVz                  $10.00           361
                                  $16.99           343
                                  $30.00           339
                                  $28.00           337
                                  $49.00           324
Name: price_string, Length: 336, dtype: int64

In [40]:
df.groupby('category')['price_string'].value_counts().unstack().fillna(0)

price_string,$0.00,$10.00,$10.29,$10.99,$100,$100.00,$1099,$11.00,$11.30,$11.75,...,14.95,17.00,21.00,22.00,32.99,5.75,52.00,6.60,67.00,7.00
category,,,,,,,,,,,,,,,,,,,,,
Q2xvdGhpbmcgYW5kIEFjY2Vzc29yaWVz,0.0,0.0,0.0,0.0,342.0,693.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,334.0,0.0,0.0,0.0
QXJ0IHN1cHBsaWVz,0.0,331.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
QmFieWNhcmU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RWxlY3Ryb25pY3M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SG91c2Vob2xkIGFuZCBDbGVhbmluZw,3175.0,0.0,0.0,335.0,0.0,0.0,348.0,366.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SGVhbHRo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VG95cyBhbmQgR2FtZXM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1418.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VG9vbHMgYW5kIGhvbWUgaW1wcm92ZW1lbnQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YmVhdXR5IGFuZCBwZXJzb25hbCBjYXJl,4196.0,330.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
len(df['category'].unique())

14

In [27]:
avg_price = df.groupby('category')['price_string'].agg(np.mean)

TypeError: ignored